## Rennes Data Challenge 2023 

Mathis Derenne

---

### Linear model

Linear model provide a forecasting method that should be better than baseline method. It uses the previous value of the time series as well as exogenous features to predict the next value.

In [28]:
import pandas as pd
import numpy as np
from utils import load_data, validation_split

In [29]:
X, y = load_data()
X['date'] = X.index

### Target feature engineering

In [30]:
def get_lag(y, window_lag=1):
    df = pd.DataFrame()
    for i in range(1, window_lag + 1):
        df[f'y_lag_{i}'] = y.shift(i)
    return df

lag = get_lag(y, window_lag=1)

In [31]:
def get_moving_average(y, moving_average=1):
    df = pd.DataFrame()
    df['y_ma'] = y.rolling(moving_average).mean().shift(1)
    return df

ma = get_moving_average(y, moving_average=3)

In [32]:
# index of rows that contain NaN
idx = lag.index[lag.isna().any(axis=1)]
idx = idx.union(ma.index[ma.isna().any(axis=1)])

# add lag and moving average to X
X = X.join(lag).join(ma)
X

# drop rows
X = X.drop(idx)
y = y.drop(idx)

# get columns name of lag + moving average
y_extracted_col = list(lag.columns) + list(ma.columns)

In [33]:
X_train = X[X.index < validation_split]
y_train = y[y.index < validation_split]
X_test = X[X.index >= validation_split]
y_test = y[y.index >= validation_split]

### Preprocessing

In [34]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

class DropMissing(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold

    def fit(self, X, y=None):
        missing_ratio = X.isnull().mean()
        self.to_drop = missing_ratio[missing_ratio > self.threshold].index
        return self

    def transform(self, X, y=None):
        return X.drop(self.to_drop, axis=1)

In [35]:
class AddDayOfWeek(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X['day_of_week'] = X['date'].dt.dayofweek
        # Convertir en distance angulaire
        X['day_of_week_sin'] = np.sin(2 * np.pi * X['day_of_week']/7)
        X['day_of_week_cos'] = np.cos(2 * np.pi * X['day_of_week']/7)
        X = X.drop(['date', 'day_of_week'], axis=1)
        return X
    
AddDayOfWeek().fit_transform(X[['date']]).head()

,day_of_week_sin,day_of_week_cos
date,,
2017-08-23,0.974928,-0.222521
2017-08-24,0.433884,-0.900969
2017-08-25,-0.433884,-0.900969
2017-08-28,0.000000,1.000000
2017-08-29,0.781831,0.623490


In [36]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA

numeric_transformer = Pipeline(steps=[
    ('drop_missing', DropMissing(threshold=0.25)),
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler()),
    ('pca', PCA(n_components=0.7, svd_solver='full'))
])

date_transformer = Pipeline(steps=[
    ('add_day_of_week', AddDayOfWeek())
])

In [37]:
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('y_col', StandardScaler(), y_extracted_col),
        ('date', date_transformer, ['date'])
    ], remainder='drop')

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('drop_missing',
                                                  DropMissing(threshold=0.5)),
                                                 ('imputer', KNNImputer()),
                                                 ('scaler', RobustScaler()),
                                                 ('pca',
                                                  PCA(n_components=0.7,
                                                      svd_solver='full'))]),
                                 ['btc_tweet_count', 'eth_tweet_count',
                                  'bnb_tweet_count', 'xrp_tweet_count',
                                  'matic_tweet_count', 'ada_tweet_count',
                                  'doge_tweet_count', 'shib_tweet_count',
                                  '...
                                  'eth_textblob_polarity_mean',
                                  'eth_vader_polarity_compound_min',
                                  'eth_vader_polarity_compound_max',
                                  'eth_vader_polarity_compound_mean',
                                  'bnb_posts_count',
                                  'bnb_textblob_polarity_min',
                                  'bnb_textblob_polarity_max',
                                  'bnb_textblob_polarity_mean', ...]),
                                ('y_col', StandardScaler(),
                                 ['y_lag_1', 'y_ma']),
                                ('date',
                                 Pipeline(steps=[('add_day_of_week',
                                                  AddDayOfWeek())]),
                                 ['date'])])

### Linear model

In [38]:
from sklearn.linear_model import Ridge

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha = 1))
])

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('drop_missing',
                                                                   DropMissing(threshold=0.5)),
                                                                  ('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=0.7,
                                                                       svd_solver='full'))]),
                                                  ['btc_tweet_count',
                                                   'eth_tweet_count',
                                                   'bnb_tweet_count',
                                                   'xrp_tweet_count',
                                                   'matic_tweet_count',
                                                   'ada_tweet_count',
                                                   'doge_t...
                                                   'eth_vader_polarity_compound_min',
                                                   'eth_vader_polarity_compound_max',
                                                   'eth_vader_polarity_compound_mean',
                                                   'bnb_posts_count',
                                                   'bnb_textblob_polarity_min',
                                                   'bnb_textblob_polarity_max',
                                                   'bnb_textblob_polarity_mean', ...]),
                                                 ('y_col', StandardScaler(),
                                                  ['y_lag_1', 'y_ma']),
                                                 ('date',
                                                  Pipeline(steps=[('add_day_of_week',
                                                                   AddDayOfWeek())]),
                                                  ['date'])])),
                ('regressor', Ridge(alpha=1))])

In [48]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'preprocessor__num__drop_missing__threshold': [0, .1, .2, .3, .4],
    'preprocessor__num__imputer__n_neighbors': [3, 5, 7],
    'preprocessor__num__pca__n_components': [.3, .4, .5, .6, .7, .8],
    'regressor__alpha': np.logspace(-4, 4, 5)
}

search = RandomizedSearchCV(model, param_grid, cv=4, n_iter = 30)

search.fit(X_train, y_train)

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.965):
{'regressor__alpha': 0.01, 'preprocessor__num__pca__n_components': 0.6, 'preprocessor__num__imputer__n_neighbors': 5, 'preprocessor__num__drop_missing__threshold': 0}


### Best model selection

In [49]:
# Get the best model
model = search.best_estimator_
pred = model.predict(X_test)
# print R2, MAE, MSE
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
print(f"R2: {r2_score(y_test, pred)}")
print(f"MAE: {mean_absolute_error(y_test, pred)}")
print(f"MSE: {mean_squared_error(y_test, pred)}")

# Make pred into a dataframe with y_test index
pred = pd.DataFrame(pred, index=y_test.index, columns=['Close_BTC'])

pred.to_csv('prediction/linear_pred.csv')

R2: 0.9346979043389085
MAE: 479.1145876725745
MSE: 567852.9283983577


### Backtesting strategy

In [57]:
def day_to_day_pred(X, y, model):
    """Fit model on every data before predicting y(t), repeat from validation_split to end of data"""
    pred = []
    n = len(X[X.index < validation_split])
    for i in range(n, len(X)):
        X_train = X.iloc[:i]
        y_train = y.iloc[:i]
        model.fit(X_train, y_train)
        pred.append(model.predict(X.iloc[i:i+1]))
    pred = np.concatenate(pred)
    # Make it into a dataframe with y index
    pred = pd.DataFrame(pred, index=y.index[n:], columns=['Close_BTC'])
    return pred

pred = day_to_day_pred(X, y, model)

In [59]:
print(f"R2: {r2_score(y_test, pred)}")
print(f"MAE: {mean_absolute_error(y_test, pred)}")
print(f"MSE: {mean_squared_error(y_test, pred)}")

R2: 0.9334248719497592
MAE: 486.3601855288957
MSE: 578922.9432701027
